In [ ]:
!pip install pyspark #pyspark is installed

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
from pyspark import SparkContext, SparkConf #packages are imported.

In [ ]:
conf = SparkConf()
conf.setMaster('local')
conf.setAppName('CTFIDF_query')
sc = SparkContext(conf=conf) #spark is necessiated.

In [ ]:
from pyspark.rdd import RDD
file2="/content/CTFIDF_index" 
import os
filename = os.path.basename(file2) #will take file name from the path.

if str(filename) != "CTFIDF_index":
  print("For this task you need to give only CTFIDF_index file , please give that file only") # if filename is not CTFIDF_index program will be executed.
  exit()
else:
  fileRDD = sc.textFile(file2) #parallelizing file to rdd.
  def conversion_to_rdd(x): #this process is used to convert back to rdd.
    y = x.split('@') #as we have every line wrod@doc-id1#freq1..... splitting on @ to get word in first element of list and remaining in second part.
    z = y[1].split('+') # Again splitting on + as to represent 2 different documents.
    document_dictionary ={} #initialinzing dictionary for storing doc-id as key and freq as value.
    for t in z :
      n = t.split('#')
      document_dictionary[n[0]] = float(n[1]) #making freq to floats for further processing 

    return (y[0],document_dictionary) #returning a tuple in which first element is word and second element is dictionary of docids as keys and freq as values.

  RDD = fileRDD.map(lambda x : conversion_to_rdd(x))
  #RDD.take(1)
  ID =0
  while True: #infinite loop untill user says quits is given.
    
    Query = input('Enter your query for CTFIDF :- ')

    from tkinter.constants import X
    import re
    import string #necessary packages are imported.
    import nltk
    nltk.download('stopwords')
    from nltk.corpus import stopwords
    stop_words = list(stopwords.words('english')) #stopwords list is created for removal .
    def preprocessing(x):
      x = x.lower()  #query words are lowered 
      array1 = x.split(" ") #splitted based on empty space.
      y = ""
      for x1 in array1: 
        if x1 in stop_words: #stopwords are removed if they are in the list of stopwords.
          continue
        else:
          y = y+ " "+ x1 + " "

      urls = re.compile(r'https?://\S+|www\.\S+')
      text_without_urls = urls.sub('', y) #query urls are removed

      text_without_punctuations = re.sub(f"[{re.escape(string.punctuation)}]", " ", text_without_urls) #https://www.geeksforgeeks.org/python-remove-punctuation-from-string/ #query punctuations are removed.

      useful_words = set()
      array2 = text_without_punctuations.split(" ")
      for x1 in array2:
        if len(x1)>0:  #I used set to add filtered words.
          useful_words.add(x1)

      return useful_words #that set is returned.


    words  = preprocessing(Query) #query is preprocessed.

    CTFIDFRDD = RDD.filter(lambda x : x[0] in words) #filtering our rdd only when words are in the set.

    import math
    def construction_of_IDF_and_TFIDF(x):
      no_of_documents_with_one_word = len(x[1]) #length of dictionary to calculate IDF which gives you no of documents the word is present.
      top =10000/(1+ no_of_documents_with_one_word) # here idf is calculated for every word and 10000 is standard total documents which is given in the question.
      IDF = math.log(top,10) 

      output = []
      for y in x[1]:
        TF_IDF_of_particular_document = x[1][y] * IDF
        per_document = (y,IDF,TF_IDF_of_particular_document) #for every value TFIDF is calculated  and overridden in dictionary.
        output.append(per_document)
      return tuple(output) #returning as tuple of list of tuples in the form (doc-id,IDF,TFIDF OF every document)


    CTFIDFRDD = CTFIDFRDD.map(lambda x : construction_of_IDF_and_TFIDF(x)) #rdd will be of the tuple of tuple form (((doc-id,IDF,TFIDF),(doc-id2,IDF2)...))
    def conversion_to_dictionary(x):
      output ={} #we get every line as (((doc-id,IDF,TFIDF),(doc-id2,IDF2)...))
      for y in x :
        values=(y[1],y[2])
        output[y[0]] = values
      return output #we wrill return dictionary in which elements will be of the form {doc-id:(IDF,TFIDF)}

    CTFIDFRDD = CTFIDFRDD.map(lambda x : conversion_to_dictionary(x)) #rdd is mapped based on the return values of convert_to_dictionary function.
    CTFIDFRDD = CTFIDFRDD.flatMap(lambda x: [(doc_id, x[doc_id]) for doc_id  in x]) #As we have every element of the form tuple of the form (word,{doc-id1:(IDF,TFIDF)}...) But our motive is to get documents so iterating in lambda function and taking that key,value pair as tuple and flattening the rdd.
    CTFIDFRDD = CTFIDFRDD.groupByKey() ## for same doc-id as key we group all tuples of the form (IDF,TFIDF) 
    CTFIDFRDD = CTFIDFRDD.mapValues(list) # we make the iterator of every key to be list.
    def Dot_product(x):
      dot_product = 0
      for y in x[1]:
        ct   = y[0]
        ctfd = y[1]
        dot_product += ct*ctfd #performing dot product and adding all the vectors.
      return (x[0],dot_product)

    CTFIDFRDD = CTFIDFRDD.map(lambda x : Dot_product(x)) #we get rdd of elements of type (doc-id, dot producted value) after calling the function.

    OutputRDD = CTFIDFRDD.sortBy(lambda x: x[1], ascending=False) #from pyspark documentation ##from pyspark documentation #using lambda function based on values sorted in descending order.
    OutputRDD = OutputRDD.zipWithIndex() # To get first 10 elements of RDD without getting out of rdd used zipindex which gives indexing for every elements.
    OutputRDD = OutputRDD.filter(lambda x : x[1]<=9) # taking first 10 document ids based on ids.
    OutputRDD = OutputRDD.map(lambda x : (x[0][0],x[0][1])) #taking as tuples.

    ID +=1 #incrementing ID because for every ID we need folder.
    foldername = "q"+str(ID)
    #os.mkdir(foldername)
    
    #OutputRDD.saveAsTextFile(folder_path)
    

    root_path = "/content/"  
    OutputRDD.saveAsTextFile(root_path + "/" + foldername) #for the root path we will add which id query is that.

    #global ID 
    


    user_input = input("Type 'quit' to exit: ") #when user wishes to quit. he should type quit.
    if user_input == "quit":
        break


Enter your query for CTFIDF :- JERUSALEM in muslims hates NA3zi in www.google.com


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Type 'quit' to exit: start
Enter your query for CTFIDF :- war regarding beer is always hating


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Type 'quit' to exit: start
Enter your query for CTFIDF :- hello seige war is cold blooded


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Type 'quit' to exit: quit
